<h1>Modelling</h1>

<h3>Importing Model Ready Data</h3>

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd
import joblib
import pickle

model_tweets = pd.read_csv('../data/model_ready_data.csv')
model_tweets = model_tweets.fillna("")
model_tweets.head()

,sentiment,lang,hashtags,clean_text
0,1,en,,rt telglobalhealth africa is in the midst of a...
1,1,en,,rt globalhlthtwit dr moeti is head of who in a...
2,1,en,red4research,rt nhsrdforum thank you research note for crea...
3,1,en,,rt highwiretalk former pfizer vp and virologis...
4,1,en,,rt peterhotez i think it s important that we d...


In [3]:
model_tweets.shape

(6417, 4)

In [4]:
# 4492 1925
sentiment_analysis_tweet_data = model_tweets.copy(deep=True)
sentiment_analysis_tweet_data.drop(sentiment_analysis_tweet_data[sentiment_analysis_tweet_data['sentiment'] == -1].index, inplace=True)
sentiment_analysis_tweet_data.reset_index(drop=True, inplace=True)
tweet_train = sentiment_analysis_tweet_data.iloc[:4492,]
tweet_test = sentiment_analysis_tweet_data.iloc[4493:,]

<h3>Sentiment Analysis</h3>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from joblib import dump, load # used for saving and loading sklearn objects
from scipy.sparse import save_npz, load_npz # used for saving and loading sparse matrices
from sklearn.decomposition import NMF, LatentDirichletAllocation

#### Unigram Counts

In [6]:
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(tweet_train['clean_text'].values)

CountVectorizer()

In [7]:
X_train_unigram = unigram_vectorizer.transform(tweet_train['clean_text'].values)

#### Unigram Tf-Idf

In [8]:
unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)


TfidfTransformer()

In [9]:
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)

#### Bigram Counts

In [10]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer.fit(tweet_train['clean_text'].values)

CountVectorizer(ngram_range=(1, 2))

In [11]:
X_train_bigram = bigram_vectorizer.transform(tweet_train['clean_text'].values)

#### Bigram Tf-Idf

In [12]:
bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

TfidfTransformer()

In [13]:
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

In [14]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np

In [15]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y,train_size=0.75, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)

    global_vars = globals()
    if(valid_score > global_vars['best_score']):
        global_vars['best_model'] = clf
        global_vars['best_model_name'] = title
        global_vars['best_score'] = valid_score

    print(f'{title}\nTrain score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')

In [16]:
y_train = tweet_train['sentiment'].values
y_train

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [17]:
best_model = ""
best_model_name = ""
best_score = 0

train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

Unigram Counts
Train score: 1.0 ; Validation score: 0.98

Unigram Tf-Idf
Train score: 1.0 ; Validation score: 0.97

Bigram Counts
Train score: 1.0 ; Validation score: 0.98

Bigram Tf-Idf
Train score: 1.0 ; Validation score: 0.98



In [18]:
print(f'The best Model is {best_model_name} with a Validation score of: {round(best_score, 2)}')

The best Model is Bigram Counts with a Validation score of: 0.98


Testing

In [19]:
def run_test_using_model(best_model: SGDClassifier, model_type: str):
    unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
    unigram_vectorizer.fit(tweet_test['clean_text'].values)
    X_test_unigram = unigram_vectorizer.transform(tweet_test['clean_text'].values)

    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
    bigram_vectorizer.fit(tweet_test['clean_text'].values)
    X_test_bigram = bigram_vectorizer.transform(tweet_test['clean_text'].values)

    y_test = tweet_test['sentiment'].values

    if(model_type == "Unigram Counts"):
        X_test = X_test_unigram

    elif(model_type == "Unigram Tf-Idf"):
        unigram_tf_idf_transformer = TfidfTransformer()
        unigram_tf_idf_transformer.fit(X_test_unigram)
        X_test_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_test_unigram)

        X_test = X_test_unigram_tf_idf

    elif(model_type == "Bigram Counts"):
        X_test = X_test_bigram

    else:
        bigram_tf_idf_transformer = TfidfTransformer()
        bigram_tf_idf_transformer.fit(X_test_bigram)

        X_test_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_test_bigram)
        X_test = X_test_bigram_tf_idf

   
    return best_model.score(X_test,y_test)


In [20]:
# type(tweet_test['clean_text'].values)
# run_test_using_model(best_model, best_model_name)

Saving generated Topic LDA Model

In [21]:
sgd = joblib.dump(best_model, '../trained_models/newsentimentSGDmodel.jl')
# then reload it with
# lda_model = joblib.load('lda_model.jl')

<h3>Topic Modelling</h3>

In [22]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\milky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

C:\Python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Python38\lib\site-packages\ipykernel\pylab\config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


Stopwords Preparation

In [24]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading Dataset

In [25]:
topic_model_data = model_tweets.copy(deep=True)
topic_model_data

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentiment,lang,hashtags,clean_text
0,1,en,,rt telglobalhealth africa is in the midst of a...
1,1,en,,rt globalhlthtwit dr moeti is head of who in a...
2,1,en,red4research,rt nhsrdforum thank you research note for crea...
3,1,en,,rt highwiretalk former pfizer vp and virologis...
4,1,en,,rt peterhotez i think it s important that we d...
...,...,...,...,...
6412,-1,en,,pin code kamala nehru pmcgvaccines covaxin min...
6413,-1,en,covid19,minute interview with the inventor of mrna vac...
6414,-1,en,,rt shawajason liars you tried to load off your...
6415,-1,en,covid19,rt kalainh as of june th t amp t has administe...


Tokenizing words

In [26]:
def get_hastags_words_list():
    hashtagList = []
    for hashtags in topic_model_data.hashtags:
        if(hashtags != ""):
            hashtagList += hashtags.split(',')

    return hashtagList

hashtag = get_hastags_words_list()

data = [word for sentence in topic_model_data.clean_text for word in sentence.split(' ')]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
hashtag[:5]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['red4research', 'wecandothis', 'covid19', 'wecandothis', 'cuban']

In [28]:
data[:10]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['rt',
 'telglobalhealth',
 'africa',
 'is',
 'in',
 'the',
 'midst',
 'of',
 'a',
 'full']

In [29]:
data_words = data + hashtag
data_words = [word for word in data_words if word != '']
data_words[:5]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['rt', 'telglobalhealth', 'africa', 'is', 'in']

Creating bigram and trigram models

In [30]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
['r', 't']


Removing Stopwords, making bigrams and lemmatization

In [31]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[[]]


In [33]:
data_lemmatized = [word for word in data_lemmatized if word != []]
data_lemmatized[:5]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['telglobalhealth'], ['africa'], ['midst'], ['full'], ['blow']]

Create Dictionary and Corpus needed for Topic Modeling

In [34]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus)

], [(64, 1)], [(7, 1)], [(765, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(441, 1)], [(64, 1)], [(64, 1)], [(844, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(7, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(374, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(278, 1)], [(7, 1)], [(844, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(720, 1)], [(441, 1)], [(64, 1)], [(64, 1)], [(194, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(1335, 1)], [(1335, 1)], [(64, 1)], [(2380, 1)], [(7, 1)], [(2381, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(2396, 1)], [(524, 1)], [(64, 1)], [(49, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(92, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(64, 1)], [(

In [35]:
# Readable View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]]

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('telglobalhealth', 1)],
 [('africa', 1)],
 [('midst', 1)],
 [('full', 1)],
 [('blow', 1)],
 [('third', 1)],
 [('wave', 1)],
 [('coronavirus', 1)],
 [('head', 1)],
 [('whoafro', 1)]]

Building the topic Model

In [36]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.088*"yet" + 0.081*"doctor" + 0.048*"roger" + 0.000*"ce" + 0.000*"yard" + '
  '0.000*"baby" + 0.000*"girl" + 0.000*"guinea" + 0.000*"equatorial" + '
  '0.000*"chinaeconomy"'),
 (1,
  '0.642*"africa" + 0.091*"wave" + 0.071*"health" + 0.058*"good" + '
  '0.000*"deltavariant" + 0.000*"india" + 0.000*"israel" + 0.000*"uganda" + '
  '0.000*"dose" + 0.000*"expire"'),
 (2,
  '0.224*"public" + 0.051*"remember" + 0.007*"sell" + 0.000*"ce" + '
  '0.000*"baby" + 0.000*"girl" + 0.000*"guinea" + 0.000*"equatorial" + '
  '0.000*"chinaeconomy" + 0.000*"cute"'),
 (3,
  '0.054*"policy" + 0.054*"amazingly" + 0.000*"birth" + 0.000*"baby" + '
  '0.000*"girl" + 0.000*"guinea" + 0.000*"equatorial" + 0.000*"chinaeconomy" + '
  '0.000*"ce" + 0.000*"intense"'),
 (4,
  '0.253*"government" + 0.163*"head" + 0.051*"rise" + 0.031*"courageous" + '
  '0.000*"chinaeconomy" + 0.000*"equatorial" + 0.000*"guinea" + 0.000*"girl" + '
  '0.000*"unicefafrica" + 0.000*"baby"'),
 (5,
  '0.115*"pfizer" + 0.055*"atlanta

Evaluating trained topic model using perplexity and cherence score

In [38]:
# Compute Perplexity
perplexity_score = lda_model.log_perplexity(corpus)
print('\nPerplexity: ', perplexity_score)  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

Perplexity:  -23.87144830096047

Coherence Score:  0.7245872265477


Visualize the topic model

In [39]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.526404 -0.272529       1        1  29.586400
15     0.200265  0.147384       2        1  19.204289
1      0.170031  0.091339       3        1  11.013465
13     0.001997 -0.015487       4        1   7.732089
7      0.246144  0.318155       5        1   6.389000
4     -0.051346 -0.021307       6        1   3.138714
16    -0.061107 -0.021045       7        1   2.963679
18    -0.051006 -0.021152       8        1   2.589579
6     -0.077565 -0.018943       9        1   2.166022
2     -0.078694 -0.018642      10        1   2.002311
12    -0.079719 -0.018331      11        1   1.717549
5     -0.082902 -0.017234      12        1   1.511704
10    -0.076474 -0.019177      13        1   1.449681
17    -0.075341 -0.019419      14        1   1.388373
8     -0.076349 -0.019170      15        1   1.346756
19    -0.087173 -0.014861      16        1   1.333382
0     -0.082950 -0.017215      17        1   1.312622
11    -0.088212 -0.014016      18        1   1.184181
9     -0.088428 -0.013798      19        1   1.026993
3     -0.087577 -0.014552      20        1   0.943212, topic_info=           Term         Freq         Total Category  logprob  loglift
64        covid  21233.00000  21233.000000  Default  30.0000  30.0000
65      vaccine  11910.00000  11910.000000  Default  29.0000  29.0000
1        africa   5119.00000   5119.000000  Default  28.0000  28.0000
7   coronavirus   3777.00000   3777.000000  Default  27.0000  27.0000
29          amp    931.00000    931.000000  Default  26.0000  26.0000
..          ...          ...           ...      ...      ...      ...
21    desperate      0.12974    357.110743  Topic20  -8.5672  -3.2566
23        thank      0.12974     44.800081  Topic20  -8.5672  -1.1808
24     research      0.12974    126.396808  Topic20  -8.5672  -2.2180
25         note      0.12974      3.870644  Topic20  -8.5672   1.2680
26       create      0.12974      3.870673  Topic20  -8.5672   1.2680

[1200 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1         3  0.999340           africa
68       20  0.909875        amazingly
29        8  0.996280              amp
59       12  0.943073          atlanta
4         9  0.989749             blow
56       11  0.947967      cdcdirector
7         5  0.998900      coronavirus
70        6  0.946638       courageous
64        1  0.999802            covid
74        2  0.965484            crazy
21        7  0.988489        desperate
71       17  0.952913           doctor
13       15  0.988736               dr
61        5  0.887075             easy
18        5  0.976792           expert
63       13  0.967273             find
3         7  0.993825             full
12       11  0.987598   globalhlthtwit
53       13  0.984405               go
15        3  0.991760             good
75        6  0.993308       government
8         6  0.989732             head
17        3  0.993319           health
73       16  0.942177        hodkinson
20        4  0.994825             know
19        4  0.994349           leader
42        4  0.980902          medical
2         4  0.994652            midst
14       14  0.979592            moeti
37       12  0.970946           pfizer
57       19  0.946226          pleased
76       20  0.909846           policy
16       10  0.988774           public
69       12  0.934971        qualified
66        9  0.950889  realjoelsmalley
60       10  0.952929         remember
24       14  0.973126         research
11        6  0.966495             rise
72       17  0.921204            roger
48       10  0.711458             sell
44       18  0.939077            speak
51        4  0.986347            still
0         1  0.552771  telglobalhealth
23        9  0.915177            thank
5         5  0.985123            third
34        4  0.996979            today
65        2  0.9997

Saving generated Topic LDA Model

In [40]:
joblib.dump(lda_model, '../trained_models/newtopicLDAmodel.jl')
# then reload it with
# lda_model = joblib.load('lda_model.jl')s


C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['../trained_models/newtopicLDAmodel.jl']

In [41]:
description = {'sentiment_analysis':{'name':best_model_name,'score':best_score}, 'topic_modeling':{'perplexity_score':perplexity_score, 'coherence_score':coherence_lda}}
joblib.dump(description, '../trained_models/newtrainedModelsData.jl')
# # then reload it with
# lda_model = pickle.load('lda_model.pk')

C:\Python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['../trained_models/newtrainedModelsData.jl']